# Policy Iteration

In [1]:
import gym
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
print('Load Packages')

Load Packages


# FrozenLake

In [2]:
env = gym.make('FrozenLake8x8-v0')

obs_space = env.observation_space
n_state = obs_space.n
print('Observation space')
print("Total {} states".format(n_state))

act_space = env.action_space
n_act = act_space.n
print('Action space')
print("Total {} actions".format(n_act))

# TRANSITION MATRIX FOR FROZEN LAKE (ENVIRONMENT MODEL)
P = env.unwrapped.P
env.render()

[2018-04-27 20:28:18,433] Making new env: FrozenLake8x8-v0


Observation space
Total 64 states
Action space
Total 4 actions

SFFFFFFF
FFFFFFFF
FFFHFFFF
FFFFFHFF
FFFHFFFF
FHHFFFHF
FHFFHFHF
FFFHFFFG


# Define functions step by step

### Initial Policy

In [3]:
policy = np.random.uniform(size=(n_state,n_act)) # sample random number between 0 and 1
policy = policy/np.sum(policy,axis=1,keepdims=True) # make it sum to 1

np.set_printoptions(precision=3,suppress=True)
print("Initial Policy Distribution:\n")
print(policy)

Initial Policy Distribution:

[[0.172 0.652 0.124 0.052]
 [0.301 0.385 0.288 0.025]
 [0.278 0.255 0.181 0.287]
 [0.204 0.436 0.07  0.291]
 [0.167 0.53  0.243 0.06 ]
 [0.079 0.39  0.387 0.143]
 [0.331 0.262 0.03  0.377]
 [0.423 0.091 0.06  0.426]
 [0.285 0.279 0.112 0.323]
 [0.184 0.46  0.166 0.19 ]
 [0.199 0.31  0.179 0.312]
 [0.236 0.173 0.32  0.271]
 [0.085 0.573 0.028 0.313]
 [0.174 0.349 0.178 0.3  ]
 [0.29  0.044 0.345 0.321]
 [0.348 0.133 0.237 0.283]
 [0.441 0.015 0.531 0.013]
 [0.264 0.137 0.32  0.278]
 [0.369 0.292 0.311 0.027]
 [0.174 0.188 0.196 0.442]
 [0.093 0.006 0.08  0.821]
 [0.291 0.023 0.288 0.399]
 [0.303 0.226 0.274 0.196]
 [0.367 0.002 0.419 0.212]
 [0.056 0.279 0.236 0.429]
 [0.366 0.411 0.024 0.199]
 [0.331 0.25  0.374 0.045]
 [0.258 0.004 0.45  0.289]
 [0.057 0.339 0.397 0.208]
 [0.022 0.077 0.51  0.392]
 [0.456 0.011 0.371 0.163]
 [0.425 0.047 0.396 0.132]
 [0.08  0.401 0.367 0.152]
 [0.052 0.321 0.318 0.309]
 [0.232 0.308 0.195 0.265]
 [0.213 0.381 0.3   0.106

### Policy Evaluation

In [4]:
def policy_evaluation(env, policy, gamma = 0.99, epsilon = 1e-6):
    '''
    env : gym environment
    gamma : discount factor
    epsilon : terminal condition
    '''
    
    # Extract environment information
    obs_space = env.observation_space
    n_state = obs_space.n
    P = env.unwrapped.P    
    
    # Random initial
    v = np.random.uniform(size=(n_state,1))
    
    while True:
        v_prime = np.zeros((n_state,))
        for s in P.keys(): # For all states s, update v(s)
            for a in P[s].keys(): # For all actions a
                for prob, next_s, reward, done in P[s][a]: # For all possible transitions (s,a,s')
                    v_prime[s] += (reward + gamma*v[next_s])*prob*policy[s][a]    
        
        dist = np.max(np.abs(v-v_prime))
        v = v_prime
        if dist < epsilon:
            break
    return v

if __name__ == '__main__':
    print('Policy evaluation for random policy\n')
    value = policy_evaluation(env, policy, gamma = 0.99, epsilon = 1e-6)
    value_map = np.reshape(value,[8,8])
    print('Value of current policy:')
    np.set_printoptions(precision=3,suppress=True)
    print(value_map)

Policy evaluation for random policy

Value of current policy:
[[0.001 0.001 0.001 0.002 0.003 0.004 0.006 0.007]
 [0.001 0.001 0.001 0.001 0.003 0.004 0.007 0.008]
 [0.    0.    0.    0.    0.002 0.004 0.008 0.013]
 [0.    0.    0.    0.    0.001 0.    0.011 0.023]
 [0.    0.    0.    0.    0.003 0.007 0.013 0.044]
 [0.    0.    0.    0.001 0.005 0.014 0.    0.119]
 [0.    0.    0.    0.    0.    0.035 0.    0.371]
 [0.    0.    0.    0.    0.055 0.187 0.362 0.   ]]


### Policy Improvement

In [5]:
def policy_improvement(env, v, gamma = 0.99):
    obs_space = env.observation_space
    n_state = obs_space.n
    act_space = env.action_space
    n_act = act_space.n
    q = np.zeros((n_state,n_act))
    
    for s in P.keys():
        for a in P[s].keys():
            for prob, next_s, reward, done in P[s][a]:
                q[s,a] += (reward + gamma*v[next_s])*prob
                    
    policy = np.zeros((n_state,n_act))
    policy[np.arange(n_state),np.argmax(q,axis=1)] = 1
    
    return policy

if __name__ == '__main__':
    print('Policy improvement\n')
    policy = policy_improvement(env, value)
    print('Improved Policy:')
    print(policy)

Policy improvement

Improved Policy:
[[0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [0. 1. 0. 0.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [0. 0. 1. 0.]
 [0. 1. 0. 0.]
 [0. 0. 0. 1.]
 [0. 0. 1. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 0. 1.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [0. 0. 1. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [0. 0. 1. 0.]
 [1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [0. 0. 1. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [1. 0. 0. 0.]]


### Policy Iteration

In [6]:
def policy_iteration(env):
    policy = np.random.uniform(size=(n_state,n_act))
    policy = policy/np.sum(policy,axis=1,keepdims=True)

    while True:
        value = policy_evaluation(env, policy) # Evaluate value of current policy
        policy_prime = policy_improvement(env, value) # Find greedy policy

        if (policy == policy_prime).all(): # If policy doesn't change, stop
            break
        policy = policy_prime # update new policy
    
    return policy, value
    
if __name__=='__main__': 
    print("Policy Iteration")
    import time
    start = time.time()
    policy_iteration(env)
    print("Computation Time : %.2f"%(time.time() - start))
    print("Value : ")
    print(value)
    print("Policy : ")
    print(policy)

Policy Iteration
Computation Time : 5.06
Value : 
[0.001 0.001 0.001 0.002 0.003 0.004 0.006 0.007 0.001 0.001 0.001 0.001
 0.003 0.004 0.007 0.008 0.    0.    0.    0.    0.002 0.004 0.008 0.013
 0.    0.    0.    0.    0.001 0.    0.011 0.023 0.    0.    0.    0.
 0.003 0.007 0.013 0.044 0.    0.    0.    0.001 0.005 0.014 0.    0.119
 0.    0.    0.    0.    0.    0.035 0.    0.371 0.    0.    0.    0.
 0.055 0.187 0.362 0.   ]
Policy : 
[[0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [0. 1. 0. 0.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [0. 0. 1. 0.]


### Run optimal policy

In [7]:
obs = env.reset()
for t in range(10000):
    print('t = %d'%t)
    env.render()
    print('')
    action = np.random.choice(n_act, 1, p=policy[obs][:])[0]
    next_obs, reward, done, info = env.step(action)
    obs = next_obs
    if done:
        break
env.render()
env.close()

t = 0

SFFFFFFF
FFFFFFFF
FFFHFFFF
FFFFFHFF
FFFHFFFF
FHHFFFHF
FHFFHFHF
FFFHFFFG

t = 1
  (Up)
SFFFFFFF
FFFFFFFF
FFFHFFFF
FFFFFHFF
FFFHFFFF
FHHFFFHF
FHFFHFHF
FFFHFFFG

t = 2
  (Up)
SFFFFFFF
FFFFFFFF
FFFHFFFF
FFFFFHFF
FFFHFFFF
FHHFFFHF
FHFFHFHF
FFFHFFFG

t = 3
  (Up)
SFFFFFFF
FFFFFFFF
FFFHFFFF
FFFFFHFF
FFFHFFFF
FHHFFFHF
FHFFHFHF
FFFHFFFG

t = 4
  (Up)
SFFFFFFF
FFFFFFFF
FFFHFFFF
FFFFFHFF
FFFHFFFF
FHHFFFHF
FHFFHFHF
FFFHFFFG

t = 5
  (Up)
SFFFFFFF
FFFFFFFF
FFFHFFFF
FFFFFHFF
FFFHFFFF
FHHFFFHF
FHFFHFHF
FFFHFFFG

t = 6
  (Up)
SFFFFFFF
FFFFFFFF
FFFHFFFF
FFFFFHFF
FFFHFFFF
FHHFFFHF
FHFFHFHF
FFFHFFFG

t = 7
  (Right)
SFFFFFFF
FFFFFFFF
FFFHFFFF
FFFFFHFF
FFFHFFFF
FHHFFFHF
FHFFHFHF
FFFHFFFG

t = 8
  (Right)
SFFFFFFF
FFFFFFFF
FFFHFFFF
FFFFFHFF
FFFHFFFF
FHHFFFHF
FHFFHFHF
FFFHFFFG

t = 9
  (Right)
SFFFFFFF
FFFFFFFF
FFFHFFFF
FFFFFHFF
FFFHFFFF
FHHFFFHF
FHFFHFHF
FFFHFFFG

t = 10
  (Right)
SFFFFFFF
FFFFFFFF
FFFHFFFF
FFFFFHFF
FFFHFFFF
FHHFFFHF
FHFFHFHF
FFFHFFFG

t = 11
  (Right)
SFFFFFFF
FFFFFFFF
FFFHFFFF
FFF

# Numpy Version

In [8]:
def policy_evaluation_np(env, P, r, policy, gamma = 0.99, epsilon = 1e-3):
    
    # Extract environment information
    obs_space = env.observation_space
    act_space = env.action_space
    n_state = obs_space.n
    n_act = act_space.n
        
    # Random initial
    v = np.random.uniform(size=(n_state,))
    
    while True:
        v_prime = np.sum(policy*np.sum((r+gamma*np.tile(v[np.newaxis,np.newaxis,:],reps=(n_state,n_act,1)))*P,axis=2),axis=1)
        dist = np.max(np.abs(v-v_prime))
        v = v_prime
        if dist < epsilon:
            break
    return v

def policy_improvement_np(env, P, r, v, gamma = 0.99):
    obs_space = env.observation_space
    act_space = env.action_space
    n_state = obs_space.n
    n_act = act_space.n
    
    q = np.sum((r+gamma*np.tile(v[np.newaxis,np.newaxis,:],reps=(n_state,n_act,1)))*P,axis=2)
                    
    policy = np.zeros((n_state,n_act))
    policy[np.arange(n_state),np.argmax(q,axis=1)] = 1
    
    return policy

def policy_iteration_np(env):
    policy = np.random.uniform(size=(n_state,n_act))
    policy = policy/np.sum(policy,axis=1,keepdims=True)
    P = np.zeros((n_state,n_act,n_state))
    r = np.zeros((n_state,n_act,n_state))
    for s in env.unwrapped.P.keys(): # For all states s, update v(s)
        for a in env.unwrapped.P[s].keys(): # For all actions a
            for prob, next_s, reward, done in env.unwrapped.P[s][a]: # For all possible transitions (s,a,s')
                P[s][a][next_s]=prob
                r[s][a][next_s]=reward
            
    while True:
        value = policy_evaluation_np(env, P, r, policy) # Evaluate value of current policy
        policy_prime = policy_improvement_np(env, P, r, value) # Find greedy policy

        if (policy == policy_prime).all(): # If policy doesn't change, stop
            break
        policy = policy_prime # update new policy
    
    return policy, value

if __name__=='__main__': 
    print("Policy Iteration Compact")
    import time
    start = time.time()
    policy, value = policy_iteration_np(env)
    print("Computation Time : %.2f"%(time.time() - start))
    print('')
    
    obs = env.reset()
    for t in range(10000):
        print('t = %d'%t)
        env.render()
        print('')
        action = np.random.choice(n_act, 1, p=policy[obs][:])[0]
        next_obs, reward, done, info = env.step(action)
        obs = next_obs
        if done:
            break
    env.render()
    env.close()

Policy Iteration Compact
Computation Time : 0.42

t = 0

SFFFFFFF
FFFFFFFF
FFFHFFFF
FFFFFHFF
FFFHFFFF
FHHFFFHF
FHFFHFHF
FFFHFFFG

t = 1
  (Down)
SFFFFFFF
FFFFFFFF
FFFHFFFF
FFFFFHFF
FFFHFFFF
FHHFFFHF
FHFFHFHF
FFFHFFFG

t = 2
  (Down)
SFFFFFFF
FFFFFFFF
FFFHFFFF
FFFFFHFF
FFFHFFFF
FHHFFFHF
FHFFHFHF
FFFHFFFG

t = 3
  (Down)
SFFFFFFF
FFFFFFFF
FFFHFFFF
FFFFFHFF
FFFHFFFF
FHHFFFHF
FHFFHFHF
FFFHFFFG

t = 4
  (Up)
SFFFFFFF
FFFFFFFF
FFFHFFFF
FFFFFHFF
FFFHFFFF
FHHFFFHF
FHFFHFHF
FFFHFFFG

t = 5
  (Up)
SFFFFFFF
FFFFFFFF
FFFHFFFF
FFFFFHFF
FFFHFFFF
FHHFFFHF
FHFFHFHF
FFFHFFFG

t = 6
  (Up)
SFFFFFFF
FFFFFFFF
FFFHFFFF
FFFFFHFF
FFFHFFFF
FHHFFFHF
FHFFHFHF
FFFHFFFG

t = 7
  (Right)
SFFFFFFF
FFFFFFFF
FFFHFFFF
FFFFFHFF
FFFHFFFF
FHHFFFHF
FHFFHFHF
FFFHFFFG

t = 8
  (Right)
SFFFFFFF
FFFFFFFF
FFFHFFFF
FFFFFHFF
FFFHFFFF
FHHFFFHF
FHFFHFHF
FFFHFFFG

t = 9
  (Right)
SFFFFFFF
FFFFFFFF
FFFHFFFF
FFFFFHFF
FFFHFFFF
FHHFFFHF
FHFFHFHF
FFFHFFFG

t = 10
  (Right)
SFFFFFFF
FFFFFFFF
FFFHFFFF
FFFFFHFF
FFFHFFFF
FHHFFFHF
FHFFHFHF
F